In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

!pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

In [ ]:
# Get single image that blends all RGBY into RGB
# Introduce the images as arrays. Can use the function above.

def get_blended_image(images): 
    # get rgby images for sample

    # blend rgby images into single array
    blended_array = np.stack(images[:-1], 2)

    # Create PIL Image
    blended_image = Image.fromarray( np.uint8(blended_array) )
    return blended_image

def image_to_arrays(path):
    
    image_arrays = list()
    for image in path:
        array = np.asarray(Image.open(image))
        image_arrays.append(array)
        
    return image_arrays

In [ ]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

NUC_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth"
CELL_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth"
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    # NOTE: setting padding=True seems to solve most issues that have been encountered
    #       during our single cell Kaggle challenge.
    padding=False,
    multi_channel_model=True,
)

def get_masks(imgs, test=True):
    try:
        images = [[img[:, :, 0] for img in imgs], 
                  [img[:, :, 3] for img in imgs], 
                  [img[:, :, 2] for img in imgs]]
    
        nuc_segmentations = segmentator.pred_nuclei(images[2])
        cell_segmentations = segmentator.pred_cells(images)
        cell_masks = []
        for i in tqdm(range(len(cell_segmentations)), desc='Labeling cells..'):
            _, cell_mask = label_cell(nuc_segmentations[i], cell_segmentations[i])
            cell_masks.append(cell_mask)
        return cell_masks
    except:
        raise ValueError('Segmentation failed')

In [ ]:
channels = ['_red.png', '_blue.png', '_yellow.png', '_green.png']
train_label = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')
train_data = '../input/hpa-single-cell-image-classification/train'
paths = [[os.path.join(train_data, train_label.iloc[idx,0])+ channel for channel in channels] for idx in range(len(train_label))]

In [ ]:
image = paths[0]
array = image_to_arrays(image)
blended_image = get_blended_image(array)
plt.imshow(blended_image)

In [ ]:
nuclei = array[1]
cell = array[:-1]
inter_step = [[i] for i in image[:-1]]

# For nuclei
nuc_segmentations = segmentator.pred_nuclei([nuclei])

# For full cells
cell_segmentations = segmentator.pred_cells(inter_step)

In [ ]:
# post-processing
nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
plt.imshow(cell_mask)

In [ ]:
# Unique vector of cell_mask numbers
numbers = set(np.ravel(cell_mask))
numbers.remove(0)

fig = plt.figure(figsize=(25,6*len(numbers)/4))
index = 1

for number in numbers:
    isolated_cell = np.where(cell_mask==number, cell_mask, 0)
    ax = fig.add_subplot(len(numbers)//4+1, 4, index)
    ax.set_title(number, size=20)
    plt.imshow(isolated_cell)
    index += 1

In [ ]:
isolated_cell = np.where(cell_mask==1, cell_mask, 0)
plt.imshow(isolated_cell)

In [ ]:
isolated_cell = np.where(cell_mask==2, cell_mask, 0)
plt.imshow(isolated_cell)